<a href="https://colab.research.google.com/github/mdjamina/model_relatives_psr_fr/blob/text_class_model/src/load_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

# Préparation des données

## Chargement des données

In [29]:
import pandas as pd
#file_path = '../data/relatives.xlsx'
file_path = 'relatives.xlsx'
sheet_name = 'data'
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [15]:
df.head()

,texts,PSR,annotation
0,"Survivante de la Shoah, elle a su puiser, dans...",qui ont défiguré notre continent et avili l’hu...,det|det
1,la Constitution permet au Président de la Répu...,"qui, de cette possibilité, font une nécessité",det
2,Les heures que nous vivons sont de celles-là.,que nous vivons,det
3,Je veux aujourd’hui vous parler du mandat que ...,que le peuple nous a donné|que je veux changer...,det|det|det
4,"Pour d’autres, au nombre desquels je me range,...",desquels je me range,det


In [16]:
df.describe()

,texts,PSR,annotation
count,546,546,546
unique,532,534,16
top,dans un monde où tout va si vite et où les exi...,qui viennent,det
freq,4,3,260


## Prétraitement des données :



### Normalisation:

cette partie permet de nettoyer et uniformiser des textes en remplaçant des caractères spéciaux par leurs équivalents standards.

In [17]:
import re

def replace_characters(match: re.Match) -> str:
    char = match.group(0)
    replacements = {'’': "'",'´': "'",'`': "'",'‘': "'",'«': '"','»': '"','“': '"','”': '"','–': '-','—': '-','…': ' ',u'\xa0': ' ',}
    return replacements[char]

def normalize_text(text: str) -> str:
    pattern = r'[’´`‘«»“”–—…]'
    return re.sub(pattern, replace_characters, text).strip()

In [18]:
# test
text = "un text avec des caractères spéciaux ’´`‘«»“”–—…"
print(normalize_text(text))

un text avec des caractères spéciaux ''''""""--


### Tokenization:

Cette partie permet de découper le texte en mots

In [19]:
# chargement de la librairie nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**RegexpTokenizer** : permet de diviser une chaîne de caractères en sous-chaînes en utilisant une expression régulière.

La fonction **tokenizer_list** permet de diviser une séquence de texte en sous-séquences en utilisant le caractère '|' comme délimiteur, puis de segmenter chaque sous-séquence en mots et ponctuations.

In [22]:
from nltk import RegexpTokenizer

def tokenizer(text):
  return RegexpTokenizer(r'''\w'|\w+|[^\w\s]''').tokenize(text)


def tokenizer_list(text):
  texts = text.split("|")
  result = []
  for t in texts:
    result.append( tokenizer(t) )

  return result

In [23]:
# Test
text = "Bonjour tout le monde!|il s'agit d'un test."
tokenizer_list(text)

[['Bonjour', 'tout', 'le', 'monde', '!'],
 ['il', "s'", 'agit', "d'", 'un', 'test', '.']]

### Labilisation:

L'objectif de cette partie est de transformer les labels en valeurs numériques et aligner les labels avec le texte tokenisé.

In [24]:
# Fonction pour annoter le texte
# permet de labiliser en mode séquentiel ou non
def annotate(text, psr_lst, tag_lst, mode_seq=False):
    # Initialiser la liste des annotations avec des 0
    annotation = [0] * len(text)


    # Définir les labels pour chaque tag
    labels = (
        {'O': 0, 'det': 1, 'appo': 3, 'ambiguë': 5} if mode_seq
        else {'O': 0, 'det': 1, 'appo': 2, 'ambiguë': 3}
    )

    # Trouver la longueur du texte
    text_len = len(text)
    if len(psr_lst) != len(tag_lst):
        raise ValueError("Les listes psr_lst et tag_lst doivent avoir la même taille")

    # Parcourir chaque psr dans psr_lst
    for i, psr in enumerate(psr_lst):
        psr_len = len(psr)  # Longueur de la sous-séquence psr

        tag = tag_lst[i]  # Etiquette correspondante

        # Parcourir le texte pour trouver des correspondances avec psr
        for j in range(text_len - psr_len + 1):
            if text[j:j + psr_len] == psr:
                annotation[j] = labels[tag] # Labelliser le token correspondant
                if mode_seq: # Si le mode séquentiel est activé, labelliser les tokens suivants
                  for k in range(j + 1, j + psr_len):
                    annotation[k] = labels[tag] + 1
                break

    return annotation

In [25]:
words = tokenizer("Vous passerez, nous passerons sans doute des jours et des nuits sur des sujets qui touchent le quotidien de nos concitoyens et qui auront chacun leur importance.")
psr = tokenizer_list("qui touchent le quotidien de nos concitoyens|qui auront chacun leur importance")
tag = 'appo|appo'.split('|')

print(words, len(words))
print(psr)
print(tag)

['Vous', 'passerez', ',', 'nous', 'passerons', 'sans', 'doute', 'des', 'jours', 'et', 'des', 'nuits', 'sur', 'des', 'sujets', 'qui', 'touchent', 'le', 'quotidien', 'de', 'nos', 'concitoyens', 'et', 'qui', 'auront', 'chacun', 'leur', 'importance', '.'] 29
[['qui', 'touchent', 'le', 'quotidien', 'de', 'nos', 'concitoyens'], ['qui', 'auront', 'chacun', 'leur', 'importance']]
['appo', 'appo']


In [26]:
# test de la fonction annotate

labels = annotate(words, psr,tag )
labels_seq = annotate(words, psr,tag, mode_seq=True)
label_names = ['O', 'DET', 'APPO', 'AMBIGUE']
label_names_seq = ['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE','I-AMBIGUE']


line1 = ""
line2 = ""
line3 = ""

for word, label, label_seq in zip(words, labels, labels_seq):
    full_label = label_names[label]
    full_label_seq = label_names_seq[label_seq]

    max_length = max(len(word), len(full_label), len(full_label_seq))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
    line3 += full_label_seq + " " * (max_length - len(full_label_seq) + 1)

# Affichage aligné des tokens et de leurs tags
print(line1)
print(line2)
print(line3)

Vous passerez , nous passerons sans doute des jours et des nuits sur des sujets qui    touchent le     quotidien de     nos    concitoyens et qui    auront chacun leur   importance . 
O    O        O O    O         O    O     O   O     O  O   O     O   O   O      APPO   O        O      O         O      O      O           O  APPO   O      O      O      O          O 
O    O        O O    O         O    O     O   O     O  O   O     O   O   O      B-APPO I-APPO   I-APPO I-APPO    I-APPO I-APPO I-APPO      O  B-APPO I-APPO I-APPO I-APPO I-APPO     O 


### Traitement

Ainsi, nous allons remplacer les caractères spéciaux par leurs équivalents standards.

In [30]:
# apply to the dataframe
df = df.applymap(normalize_text)

Application de la tokenization sur les données

In [31]:
labels = {'O': 0, 'det': 1, 'appo': 2, 'ambiguë': 3}


In [32]:
# Application de la fonction tokenizer sur les colonnes Phrase et PSR
df['tokens'] = df['texts'].apply(tokenizer)
df['tokens_psr'] = df['PSR'].apply(tokenizer_list)
df['PSR'] = df['PSR'].apply(lambda x : [t.strip() for t in x.split("|")])
df['annotation'] = df['annotation'].apply(lambda x : [t.strip() for t in x.split("|")])
df['labels'] = df['annotation'].apply(lambda x : labels[list(set(x))[0]] )

In [33]:
df.head()

,texts,PSR,annotation,tokens,tokens_psr,labels
0,"Survivante de la Shoah, elle a su puiser, dans...",[qui ont défiguré notre continent et avili l'h...,"[det, det]","[Survivante, de, la, Shoah, ,, elle, a, su, pu...","[[qui, ont, défiguré, notre, continent, et, av...",1
1,la Constitution permet au Président de la Répu...,"[qui, de cette possibilité, font une nécessité]",[det],"[la, Constitution, permet, au, Président, de, ...","[[qui, ,, de, cette, possibilité, ,, font, une...",1
2,Les heures que nous vivons sont de celles-là.,[que nous vivons],[det],"[Les, heures, que, nous, vivons, sont, de, cel...","[[que, nous, vivons]]",1
3,Je veux aujourd'hui vous parler du mandat que ...,"[que le peuple nous a donné, que je veux chang...","[det, det, det]","[Je, veux, aujourd, ', hui, vous, parler, du, ...","[[que, le, peuple, nous, a, donné], [que, je, ...",1
4,"Pour d'autres, au nombre desquels je me range,...",[desquels je me range],[det],"[Pour, d', autres, ,, au, nombre, desquels, je...","[[desquels, je, me, range]]",1


#### vérification de la tokenization

In [34]:
# Fonction pour vérifier la taille des listes dans les colonnes 'PSR' et 'annotation'
def check_list_sizes(row):
    psr_len = len(row['PSR'])
    annotation_len = len(row['annotation'])
    return psr_len == annotation_len

In [35]:
# Appliquer la vérification à chaque ligne du DataFrame
df['size_match'] = df.apply(check_list_sizes, axis=1)

In [36]:
# Supprimer des lignes où la taille des listes dans les colonnes 'PSR' et 'annotation' ne correspond pas
df = df[df['size_match'] == True]

Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags

In [37]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_tags'] = df.apply(lambda row: annotate(row['tokens'], row['PSR'], row['annotation']), axis=1)

In [38]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_seq_tags'] = df.apply(lambda row: annotate(row['tokens'], row['PSR'], row['annotation'], mode_seq=True), axis=1)

Supprimer les colonnes

In [39]:
# Supprimer les colonnes PSR et annotation

columns_to_drop = [ c for c in df.columns if c not in ['texts','tokens','psr_tags', 'psr_seq_tags', 'labels']]


print(columns_to_drop)


df.drop(columns=columns_to_drop, inplace=True)



# reinstialiser l'index
df.reset_index(names='id' ,inplace=True)

['PSR', 'annotation', 'tokens_psr', 'size_match']


In [40]:

df.head()

,id,texts,tokens,labels,psr_tags,psr_seq_tags
0,0,"Survivante de la Shoah, elle a su puiser, dans...","[Survivante, de, la, Shoah, ,, elle, a, su, pu...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,la Constitution permet au Président de la Répu...,"[la, Constitution, permet, au, Président, de, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,Les heures que nous vivons sont de celles-là.,"[Les, heures, que, nous, vivons, sont, de, cel...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,Je veux aujourd'hui vous parler du mandat que ...,"[Je, veux, aujourd, ', hui, vous, parler, du, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"Pour d'autres, au nombre desquels je me range,...","[Pour, d', autres, ,, au, nombre, desquels, je...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Création Dataset Hugging Face

### Découpage du jeu de données en train, validation et test

In [41]:
#split the dataset into train , validation and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [42]:
# Supprimer les colonnes d'index auto-générées si elles existent
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [43]:
train_df.head()

,id,texts,tokens,labels,psr_tags,psr_seq_tags
0,16,je demanderai au Premier ministre d'assigner à...,"[je, demanderai, au, Premier, ministre, d', as...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,263,Nos institutions sont en effet solides. La Vᵉ ...,"[Nos, institutions, sont, en, effet, solides, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,217,Nous devons faire en sorte que nos discussions...,"[Nous, devons, faire, en, sorte, que, nos, dis...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,435,"Le groupe UDI, Agir et indépendants plaide dep...","[Le, groupe, UDI, ,, Agir, et, indépendants, p...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,63,Sieyès et Mirabeau ne désertèrent pas si promp...,"[Sieyès, et, Mirabeau, ne, désertèrent, pas, s...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Préparation des datasets

In [44]:
from datasets import Dataset, Features, Sequence, Value, ClassLabel

In [45]:
# Définir les Features pour chaque colonne du DataFrame
features = Features({
    'id': Value(dtype='int32', id=None),
    'texts': Value(dtype='string', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'labels': ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None),
    'psr_tags': Sequence(feature=ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None), length=-1, id=None),
    'psr_seq_tags': Sequence(feature=ClassLabel(names=['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE','I-AMBIGUE'], id=None), length=-1, id=None)
})

In [46]:
# Convertir le DataFrame en Dataset Hugging Face avec les Features spécifiés
train_ds = Dataset.from_pandas(train_df, features=features)
val_ds = Dataset.from_pandas(val_df, features=features)
test_ds = Dataset.from_pandas(test_df, features=features)

In [47]:
train_ds

Dataset({
    features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
    num_rows: 392
})

In [48]:
from datasets import DatasetDict

# Recréation du DatasetDict
dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})

In [49]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 99
    })
    test: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 55
    })
})

### Sauvegarde des datasets et publication sur Hugging Face

In [ ]:
dataset_name = 'relatives_psr'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Enregistrer le DatasetDict dans le hub Hugging Face
dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/djamina/relatives_psr/commit/1f7c7701776a5850f65f8e7ff685d1824d6a3188', commit_message='Upload dataset', commit_description='', oid='1f7c7701776a5850f65f8e7ff685d1824d6a3188', pr_url=None, pr_revision=None, pr_num=None)